In [1]:
# Импорт библиотек
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
# Чтение датасета
df = pd.read_csv('/Users/stanislawtsurkan/ds_bootcamp/rent-real-estate/data/_data.csv')

In [5]:
# удаляем столбец `Unnamed: 0`
df = df.iloc[:, 1:]

# переименовываем столбцы
NEW_COLUMNS = {
    'ID  объявления':           'id',
    'Количество комнат':        'rooms',
    'Тип':                      'type',
    'Метро':                    'metro_station',
    'Адрес':                    'address',
    'Площадь, м2':              'area',
    'Дом':                      'building_type',
    'Парковка':                 'parking',
    'Цена':                     'price',
    'Телефоны':                 'phones',
    'Описание':                 'description',
    'Ремонт':                   'renovation',
    'Площадь комнат, м2':       'rooms_area',
    'Балкон':                   'balcony',
    'Окна':                     'windows',
    'Санузел':                  'bathroom',
    'Можно с детьми/животными': 'kids_pets',
    'Дополнительно':            'extra',
    'Название ЖК':              'complex_title',
    'Серия дома':               'building_series',
    'Высота потолков, м':       'ceiling_height',
    'Лифт':                     'elevator',
    'Мусоропровод':             'trash_chute',
    'Ссылка на объявление':     'listing_url',
}
df = df.rename(columns=NEW_COLUMNS)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23368 entries, 0 to 23367
Data columns (total 23 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   rooms            22327 non-null  object 
 1   type             23368 non-null  object 
 2   metro_station    22053 non-null  object 
 3   address          23368 non-null  object 
 4   area             23368 non-null  object 
 5   building_type    23368 non-null  object 
 6   parking          9951 non-null   object 
 7   price            23368 non-null  object 
 8   phones           23368 non-null  object 
 9   description      23368 non-null  object 
 10  renovation       20613 non-null  object 
 11  rooms_area       14458 non-null  object 
 12  balcony          15390 non-null  object 
 13  windows          16755 non-null  object 
 14  bathroom         20696 non-null  object 
 15  kids_pets        17272 non-null  object 
 16  extra            23011 non-null  object 
 17  complex_titl

In [ ]:
# 
df['area'].unique().tolist()

['200.0/20.0',
 '198.0/95.0/18.0',
 '200.0/116.0/4.0',
 '170.0/95.0/17.0',
 '58.0/38.0/5.0',
 '92.0',
 '213.0/140.0/20.0',
 '98.0/63.0/9.0',
 '120.0/95.0/10.0',
 '90.0/66.0/10.0',
 '170.0/94.0/10.0',
 '80.0/53.0/11.0',
 '100.0/80.0/10.0',
 '180.0/150.0/20.0',
 '222.2',
 '225.0',
 '30.0/20.0',
 '222.0',
 '150.0',
 '201.0/162.0/16.0',
 '64.0/50.0/8.0',
 '112.0/77.0/8.0',
 '190.0',
 '255.6/204.0',
 '186.0/158.0/20.0',
 '100.0/61.0/16.0',
 '150.0/139.0/9.0',
 '140.0/123.0/15.0',
 '132.8/106.0',
 '256.0/144.0/26.0',
 '380.0/250.0/28.0',
 '112.0',
 '91.0',
 '135.0/100.0/7.0',
 '190.0/112.0/12.0',
 '44.0',
 '85.0/68.0',
 '100.0',
 '133.3',
 '210.0',
 '180.0',
 '165.0',
 '168.0',
 '250.0',
 '85.0',
 '105.0/70.0/15.0',
 '134.0',
 '60.0/43.0/10.0',
 '142.0/81.0/17.0',
 '222.0/205.0/10.0',
 '60.0/40.0/11.0',
 '103.0/74.0/16.0',
 '170.0',
 '200.0/150.0/30.0',
 '105.0/60.0/18.0',
 '60.0/30.0/10.0',
 '155.0/97.0/14.0',
 '134.0/95.0/7.0',
 '214.6',
 '204.0/109.0/23.0',
 '180.0/123.0/4.0',
 '260.0',
 

In [ ]:
b = df['area'].apply(lambda x: x.split('/')[0])


Series([], Name: area, dtype: object)

In [10]:
b

['200.0', '116.0', '4.0']